# <span style="color:red"> Introduction to Python, Part 3 </span>

In [1]:
# Import code libraries or "modules" in Python lingo

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import sys
from datascience import *
sns.set_style("whitegrid")
%matplotlib inline

### <span style="color:blue"> Tables </span>

For illustration let's make simple table

In [2]:
flowers = Table().with_columns(
    "name", make_array("lotus", "sunflower", "rose"),
    "petals", make_array(8, 34, 5),
    "color", make_array("pink", "yellow", "red")
)
flowers

name,petals,color
lotus,8,pink
sunflower,34,yellow
rose,5,red


Rows of the table represent objects (in his case, flowers). 
Columns of the table represent features of objects (in this case, name, number of petals, color). This data format is very common; many data sets come in this format or can be converted to this format.

The datascience module has many functions for operating on tables. 

Here is a 
[cheatsheet](https://github.com/wstuetzle/STAT180/blob/master/Computing/data8_sp17_midterm_ref_sheet.pdf).

Here is a [tutorial](http://data8.org/datascience/tutorial.html)

Let's try some of the functions / methods

In [3]:
print(flowers.num_rows)       # number of rows
print(flowers.num_columns)    # number of columns

3
3


In [ ]:
flowers.labels     # names of columns

In [4]:
flowers.column("name")     # extract a column

array(['lotus', 'sunflower', 'rose'], 
      dtype='<U9')

In [5]:
flowers.select("name", 1)    

# makes a new table with only the selected columns
# columns can be identified by name or index
# remember: indexing start at 0                    

name,petals
lotus,8
sunflower,34
rose,5


In [6]:
# The table  flowers  is not changed

flowers

name,petals,color
lotus,8,pink
sunflower,34,yellow
rose,5,red


In [ ]:
# But we can define new table

flower_names_only = flowers.select("name", 1) 
flower_names_only

In [7]:
flowers.take(make_array(0, 1))    

# make a new table that has only the first two rows

name,petals,color
lotus,8,pink
sunflower,34,yellow


In [8]:
sorted_flowers = flowers.sort("petals")
sorted_flowers

# sort the column "petals" and shuffels the other columns along

name,petals,color
rose,5,red
lotus,8,pink
sunflower,34,yellow


In [9]:
flowers

# again, flowers did not change - good!

name,petals,color
lotus,8,pink
sunflower,34,yellow
rose,5,red


In [10]:
flowers.where("petals", are.above(5))    

# select rows by values of a column

name,petals,color
lotus,8,pink
sunflower,34,yellow


### <span style="color:blue"> Who survived when the Titanic sank? (Part 1) </span>

More info on the sinking of the Titanic in [Encyclopedia Titanica](http://www.encyclopedia-titanica.org) or Wikipedia

A "comma separated values" (csv) version of the dataset we will be analyzing is on Canvas. You can downoad it and look at in using Excel. Let's do that. 

The spreadsheet has 1,310 rows, but the first one is the column labels. So there are data on 1,309 passengers.

A description of the dataset is on Canvas.

**Here are the column labels and their meanings**
```
    pclass     cabin class
    survived   0 = "no", 1 = "yes"
    name
    sex
    age
    sibsp      number of siblings or spouses aboard
    parch      number of parents/children aboard
    ticket     ticket number
    fare       passenger fare (in British pounds)
    cabin      cabin number
    embarked   port of embarkation (C + cherbourg; Q = Queenstown; S = Southampton
    boat       lifeboat number
    body       body identification number
    home.dest  home/destination 
```

**Note**

If you want to run this notebook on Azure, its easiest to upload the notebook, the file "datascience_extensions.py" (on Canvas) and the file "titanic3.csv" (on Canvas) and then replace the next two cells (below) by a single code cell
with the following two lines:

from datascience_extensions import * <br>
titanic = Table.read_table("titanic3.csv")

In [ ]:
# Import some new methods for class Table
# Add IDS directory to search path
course_dir = "/Users/wxs/Dropbox/IDS/Git-reps/STAT180/"
computing_dir = course_dir + "Computing"

if computing_dir not in sys.path:
    sys.path.append(computing_dir)

from datascience_extensions import *

# Reload the extensions after we make a change
# Importing it again does not work - a module is imported only once
module_name = "datascience_extensions"
importlib.reload(sys.modules[module_name])

In [ ]:
# Read the data into a table "titanic"
data_dir = course_dir + "Data/"
titanic_filename = "titanic3.csv"
titanic_pathname = data_dir + titanic_filename
titanic = Table.read_table(titanic_pathname)

In [5]:
Table.read_table('https://github.com/wstuetzle/STAT180/blob/master/Data/titanic3.csv')


ParserError: Error tokenizing data. C error: Expected 1 fields in line 36, saw 2


In [7]:
Table.read_table("http://github.com/wstuetzle/STAT180/blob/master/Data/loan-prediction-data.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 36, saw 2


In [ ]:
# Display the first three rows

titanic.take(make_array(1, 2, 3))

Rows of the table correspond to passengers <br> 
Columns correspond to properties (or "features") of passengers

We often refer to the rows as "cases" and to the columns as "variables". Unfortunate: the term "variable" is overloaded.

**Some entries in the table are "nan", indicating that the value is missing**

In [ ]:
# Let's see how big the table is

titanic.shape()

# shape is a method defined in the module  datascience_extensions

In [ ]:
# So the table "titanic" has 1,310 rows, numbered 0..1309
# The spreadsheet "titanic3.csv" has also 1,310 rows, but row 1 contains 
# the column labels. So we only have data on 1,309 passengers.
# Something is amiss. Let's look at the last ow of the table

titanic.take(1309)

In [ ]:
# "nan" is a code for "missing"
# So Table.read_table has read an empty row (all elements missing). 
# Let's get rid of that row

titanic = titanic.take(np.arange(1309))
titanic.take(1308)

# Now the last row looks legit

In [ ]:
# Complication:
# There are missing values (nan) in the table. 
# 
# Let's count the missing values for each variable
nan_count = titanic.count_nan()
nan_count

# count_nan is a method in the module datascience_extensions

### <span style="color:blue"> Dealing with missing values </span>

Missing values are a common problem in many "real world" data sets

There is no simple recipe.

One option is "imputation": guess the missing values for a row (passenger), possibly based on the values of the features that are non-missing (present).

* Guess a missing age by the average of all present ages
* Guess a missing age by the average age of passengers with the same sex, pclass, and with present age



For some questions the fact that the dataset has missing values does not matter. For example we can calculate the percentage of survivors because the value of "survived" is present for all passengers.

If a question involves features with missing values we can either throw up our hands or qualify our answer as in
"for passengers for whom we know the age, higher age goes with higher (or lower?) survival rate"

### <span style="color:blue"> Computing some simple summaries </span>

In [ ]:
# Survival rate - what fraction of passengers survived?

np.mean(titanic.column("survived"))

# Ignore the warning


In [ ]:
# To get the percentage of survivors

round(100 * np.mean(titanic.column("survived")))

In [ ]:
# What's the average age?

np.mean(titanic.column("age"))

# Whoops - age is missing for 263 passengers, so we can't compute the average age

In [ ]:
# Let's compute the average age for passengers with age present

round(np.mean(titanic.select("age").take_complete_rows().column("age")), 0)

# Let's parse this statement

In [ ]:
# Let's see if there is a difference in the average age between males and females
# and between passengers in different cabin classes

titanic_sap = titanic.select("sex", "pclass", "age").take_complete_rows()
titanic_sap.shape()

In [ ]:
round(np.mean(titanic_sap.where("sex", are.equal_to("male")).column("age")))

In [ ]:
round(np.mean(titanic_sap.where("sex", are.equal_to("female")).column("age")))

# male passengers on average are two years older than female passengers

In [ ]:
# Let's look at age versus cabin class

round(np.mean(titanic_sap.where("pclass", are.equal_to(1)).column("age")))

In [ ]:
round(np.mean(titanic_sap.where("pclass", are.equal_to(3)).column("age")))

# Passengers in 1st class are on avergage much older than passengers in 
# 3rd class. Conjecture: passengers in 3rd class are emigrants who
# tend to be young. There are other indications in the data supporting this
# conjecture